# CSL Simulation Visual Storytelling

## 1. Contributor Fund Distribution
The histogram shows the distribution of final contributor funds across a parameter sweep. 
We can see that most scenarios cluster around $X–$Y, with outliers achieving significantly higher funding due to higher corporate contributions or adopter share multipliers.

## 2. Adoption and Active Developers Over Time
The line chart highlights how CSL’s structured incentives grow both total adoption and active developer engagement. 
Steady growth in active developers aligns with higher adoption, demonstrating network effects.

## 3. Fund Component Breakdown
The grouped bar chart makes the contribution of net cash and adopter share visible alongside the total fund. 
Even small adopter shares accumulate over time and visibly increase total fund sustainability.

## 4. Key Metrics
| Metric | Mean | Std | Min | Max |
|--------|------|-----|-----|-----|
| Net Cash Contribution | ... | ... | ... | ... |
| Adopter Share | ... | ... | ... | ... |
| Total Fund | ... | ... | ... | ... |
| Total Adoption | ... | ... | ... | ... |
| Active Developers | ... | ... | ... | ... |

**Narrative:** CSL’s higher contributor share and structured barriers help grow the fund, adoption, and active developers more sustainably than MIT baseline or laissez-faire approaches. Network effects and corporate participation create compounding benefits.

In [ ]:
# -----------------------------
# Visual Storytelling Grid - Fixed & Stacked npm Charts
# -----------------------------
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

sns.set(style='whitegrid', context='talk')
plt.rcParams['figure.figsize'] = (14,6)

# -----------------------------
# 0) Load Data
# -----------------------------
display(Markdown("## Step 0: Load Data"))

df_sweep = pd.read_csv('csl_parameter_sweep.csv')
fund_csv = 'prepost_downloads_real_enriched.csv'
df_fund = pd.read_csv(fund_csv) if os.path.exists(fund_csv) else pd.read_csv('prepost_downloads_real.csv')
df_csl_path = 'outputs/res_csl.csv'
df_csl = pd.read_csv(df_csl_path) if os.path.exists(df_csl_path) else None

# -----------------------------
# 1) Funds & CSL Grid
# -----------------------------
display(Markdown("## 1) Funds & CSL Overview"))

# Determine which plots are available
plots = []
if 'final_fund' in df_sweep.columns:
    plots.append('fund_hist')
if df_csl is not None and all(col in df_csl.columns for col in ['t','total_adoption','active_developers']):
    plots.append('csl_line')
if all(col in df_fund.columns for col in ['t','net_cash_to_fund','adopter_share','contributor_fund']):
    plots.append('fund_bar')
if 'contributor_fund' in df_fund.columns:
    plots.append('fund_cum')

# Create 2x2 grid
n_rows = 2
n_cols = 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16,12))
axes = axes.flatten()

for ax, plot_type in zip(axes, plots):
    if plot_type == 'fund_hist':
        sns.histplot(df_sweep['final_fund'], bins=12, kde=True, color='tab:blue', ax=ax)
        ax.set_title("Distribution of Contributor Funds")
        ax.set_xlabel("Final Fund ($)")
        ax.set_ylabel("Frequency")
    elif plot_type == 'csl_line':
        ax.plot(df_csl['t'], df_csl['total_adoption'], label='Total Adoption', color='tab:brown', linewidth=2)
        ax.plot(df_csl['t'], df_csl['active_developers'], label='Active Developers', color='tab:purple', linewidth=2)
        ax.set_title("CSL Simulation Over Time")
        ax.set_xlabel("Time Steps")
        ax.set_ylabel("Count")
        ax.legend()
        ax.grid(True)
    elif plot_type == 'fund_bar':
        width = 0.35
        t = df_fund['t']
        ax.bar(t - width/2, df_fund['net_cash_to_fund'], width=width, label='Net Cash Contribution', color='tab:orange', alpha=0.8)
        ax.bar(t + width/2, df_fund['adopter_share'], width=width, label='Adopter Share', color='tab:blue', alpha=0.8)
        ax.plot(t, df_fund['contributor_fund'], color='black', linewidth=2, label='Total Fund')
        ax.set_title("Contributor Fund Breakdown")
        ax.set_xlabel("Time Steps")
        ax.set_ylabel("USD")
        ax.legend()
        ax.grid(True)
    elif plot_type == 'fund_cum':
        df_fund['cumulative_fund'] = df_fund['contributor_fund'].cumsum()
        ax.plot(df_fund['t'], df_fund['cumulative_fund'], color='green', linewidth=2)
        ax.set_title("Cumulative Contributor Fund")
        ax.set_xlabel("Time Steps")
        ax.set_ylabel("Cumulative USD")
        ax.grid(True)

# Remove unused axes
for ax in axes[len(plots):]:
    ax.remove()

plt.tight_layout()
plt.show()

# -----------------------------
# 2) npm Adoption Trends - Stacked
# -----------------------------
display(Markdown("## 2) npm Adoption Trends"))

# Smoothed monthly downloads (line)
if all(col in df_fund.columns for col in ['month_dt','downloads_ma3','project']):
    fig, ax = plt.subplots(figsize=(16,6))
    sns.lineplot(data=df_fund, x='month_dt', y='downloads_ma3', hue='project', ax=ax)
    ax.set_title("Smoothed Monthly npm Downloads")
    ax.set_xlabel("Month")
    ax.set_ylabel("Downloads (3-month MA)")
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    ax.grid(True)
    plt.tight_layout()
    plt.show()

# Pre/Post Event Comparison (box)
if all(col in df_fund.columns for col in ['project','downloads_ma3','post']):
    fig, ax = plt.subplots(figsize=(16,6))
    sns.boxplot(x='project', y='downloads_ma3', hue='post', data=df_fund, ax=ax)
    ax.set_title("Pre/Post Event Downloads Comparison")
    ax.set_xlabel("Project")
    ax.set_ylabel("Downloads (3-month MA)")
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    ax.legend(title='Post Event', fontsize=8)
    ax.grid(True)
    plt.tight_layout()
    plt.show()

# -----------------------------
# 3) Fund vs Adoption Scatter
# -----------------------------
display(Markdown("## 3) Fund vs Adoption"))

if all(col in df_fund.columns for col in ['contributor_fund','downloads_ma3','project']):
    plt.figure(figsize=(12,6))
    sns.scatterplot(x='contributor_fund', y='downloads_ma3', hue='project', data=df_fund, alpha=0.7)
    plt.title("Contributor Fund vs npm Downloads")
    plt.xlabel("Contributor Fund (USD)")
    plt.ylabel("Downloads (3-month MA)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
